In [ ]:
from urllib.parse import urlparse
import psycopg2
import psycopg2.extras
from pprint import pprint
import pandas as pd

import plotly.io as pio
import plotly.express as px


import os
from decouple import AutoConfig
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)
config = AutoConfig(search_path=parent_directory)


from pathlib import Path
path = os.path.abspath(os.getcwd())
output_path = path + '/dataframes/' + config("guild_id")
guild_id = config("guild_id")
try:
    path = Path(output_path)
    path.mkdir(parents=True)
except:
    print("output_path already created")

url = urlparse(config("db_url"))
connection = psycopg2.connect(
    host=url.hostname,
    port=url.port,
    database=url.path[1:],
    user=url.username,
    password=url.password
)
cursor = connection.cursor(cursor_factory = psycopg2.extras.RealDictCursor)

## What discord user has the longest average message length in a particular guild?

In [ ]:
query = f"""
select 
	authors_t.author_name,
	authors_t.nickname,
	avg_content_length_t.content_length,
	avg_content_length_t.content_count,
	authors_t.id
from 
(
	select 
		author_guild_id, 
		count(content_length) as content_count,
		AVG(content_length) as content_length
	from
		messages_t
	where
		isBot = false -- TO CHANGE
        and guild_id = '{guild_id}'
	group by author_guild_id
) as avg_content_length_t
join authors_t
on authors_t.id = avg_content_length_t.author_guild_id
order by avg_content_length_t.content_length desc;
"""

In [ ]:
cursor.execute(query)
ealiest_message_per_guild_results = cursor.fetchall()
df = pd.DataFrame.from_dict(ealiest_message_per_guild_results)

In [ ]:
df.to_csv(output_path + "/What discord user has the longest average message length in a particular guild.csv")

In [ ]:
df

## What discord user sent the most messages in a particular discord guild?

In [ ]:
query = f"""
select
	authors_t.author_name,
	authors_t.nickname,
	msg_count_per_author_t.msg_count,
	msg_count_per_author_t.author_guild_id,
	guilds_t.guild_name,
	guilds_t.id as guild_id
from
(
	select
		count(content) as msg_count,
		author_guild_id
	from
		messages_t
    where guild_id = '{guild_id}'
	group by author_guild_id
	order by msg_count desc
) as msg_count_per_author_t
join authors_t on msg_count_per_author_t.author_guild_id = authors_t.id
join guilds_t  on authors_t.guild_id = guilds_t.id
order by msg_count_per_author_t.msg_count desc;
"""

In [ ]:
cursor.execute(query)
ealiest_message_per_guild_results = cursor.fetchall()
df = pd.DataFrame.from_dict(ealiest_message_per_guild_results)

In [ ]:
df.to_csv(output_path + "/What discord user sent the most messages in a particular discord guild.csv")

## Who is the most consistent poster within a Discord Guild?

In [ ]:
query = f"""

select
	authors_t.author_name,
	authors_t.nickname,
	count_msg_day_t.day_count,
	count_msg_day_t.author_guild_id,
	guilds_t.guild_name,
	guilds_t.id as guild_id
from
(
	select
		count(distinct_msg_day_t.num_days_posted) as day_count,
		distinct_msg_day_t.author_guild_id
	from 
	(
		SELECT 
			distinct ( TO_CHAR(msg_timestamp, 'YYYY-MM-DD') )  as num_days_posted,
			author_guild_id
		from
			messages_t
        where guild_id = '{guild_id}'
	) as distinct_msg_day_t
	group by distinct_msg_day_t.author_guild_id
	order by distinct_msg_day_t.author_guild_id desc
) as count_msg_day_t
join authors_t on count_msg_day_t.author_guild_id = authors_t.id
join guilds_t on authors_t.guild_id = guilds_t.id 
order by day_count desc;

"""

In [ ]:
cursor.execute(query)
ealiest_message_per_guild_results = cursor.fetchall()
df = pd.DataFrame.from_dict(ealiest_message_per_guild_results)

In [ ]:
df.to_csv(output_path + "/Who is the most consistent poster within a Discord Guild.csv")

In [ ]:
df

## What discord author got the most reactions to their messages?

In [ ]:
query = f"""
select
	authors_t.author_name,
	authors_t.nickname, 
	reaction_count_t.reaction_count,
	guilds_t.guild_name,
	reaction_count_t.author_guild_id,
	guilds_t.id as guild_id
from
(
	select 
		author_guild_id,
		sum(count) as reaction_count
	from
		reactions_t
    where guild_id = '{guild_id}'
	group by author_guild_id
) as reaction_count_t
join authors_t on reaction_count_t.author_guild_id = authors_t.id
join guilds_t  on authors_t.guild_id = guilds_t.id
order by reaction_count desc;
"""

In [ ]:
cursor.execute(query)
ealiest_message_per_guild_results = cursor.fetchall()
df = pd.DataFrame.from_dict(ealiest_message_per_guild_results)

In [ ]:
df.to_csv(output_path + "/What discord author got the most reactions to their messages.csv")

In [ ]:
df

## What discord author got the most distinct reactions to their messages?

In [ ]:
query = f"""
select
	authors_t.author_name,
	authors_t.nickname, 
	reaction_count_t.reaction_count,
	guilds_t.guild_name,
	reaction_count_t.author_guild_id,
	guilds_t.id as guild_id
from
(
	select 
		author_guild_id,
		count(distinct(count)) as reaction_count
	from
		reactions_t
    where guild_id = '{guild_id}'
	group by author_guild_id
) as reaction_count_t
join authors_t on reaction_count_t.author_guild_id = authors_t.id
join guilds_t  on authors_t.guild_id = guilds_t.id
order by reaction_count desc;
"""

In [ ]:
cursor.execute(query)
ealiest_message_per_guild_results = cursor.fetchall()
df = pd.DataFrame.from_dict(ealiest_message_per_guild_results)

In [ ]:
df.to_csv(output_path + "/What discord author got the most distinct reactions to their messages.csv")

In [ ]:
df

## What discord user has the highest spike in activity?

In [ ]:
query = """
select 
	authors_t.author_name,
	authors_t.nickname,
	msg_date_agg_t.day_msg_count,
	msg_date_agg_t.author_guild_id,
	guilds_t.guild_name,
	guilds_t.id as guild_id
from
(
	select
		count(msg_date_t.msg_date) as day_msg_count,
		msg_date_t.msg_date,
		msg_date_t.author_guild_id
	from
	(
		SELECT
			TO_CHAR(msg_timestamp, 'YYYY-MM-DD') as msg_date,
			author_guild_id
		from
			messages_t
	) as msg_date_t
	group by msg_date_t.msg_date, msg_date_t.author_guild_id
) as msg_date_agg_t
join authors_t on msg_date_agg_t.author_guild_id = authors_t.id
join guilds_t on authors_t.guild_id = guilds_t.id
order by day_msg_count desc;
"""

In [ ]:
cursor.execute(query)
ealiest_message_per_guild_results = cursor.fetchall()
df = pd.DataFrame.from_dict(ealiest_message_per_guild_results)

In [ ]:
df.to_csv(output_path + "/What discord user has the highest spike in activity.csv")

In [ ]:
df

## What discord user was the most active for a particular date range within a particular discord guild?

In [ ]:
query = f"""
select
	authors_t.author_name,
	authors_t.nickname,
	msg_count_per_author_t.msg_count,
	msg_count_per_author_t.author_guild_id,
	guilds_t.guild_name,
	guilds_t.id as guild_id
from
(
	select
		count(content) as msg_count,
		author_guild_id
	from
		messages_t
	where
		msg_timestamp > to_date('2023-10-01', 'YYYY-MM-DD')
		and msg_timestamp < to_date('2023-10-12', 'YYYY-MM-DD') 
        and guild_id = '{guild_id}'
	group by author_guild_id
	order by msg_count desc
) as msg_count_per_author_t
join authors_t on msg_count_per_author_t.author_guild_id = authors_t.id
join guilds_t  on authors_t.guild_id = guilds_t.id
order by msg_count_per_author_t.msg_count desc;
"""

In [ ]:
cursor.execute(query)
ealiest_message_per_guild_results = cursor.fetchall()
df = pd.DataFrame.from_dict(ealiest_message_per_guild_results)

In [ ]:
df.to_csv(output_path + "/What discord user was the most active for a particular date range within a particular discord guild.csv")

In [ ]:
df

## Who sent the most attachments?

In [ ]:
query = f"""
select
	authors_t.author_name,
	authors_t.nickname,
	attachment_msg_count_t.attachment_msg_count,
	guilds_t.guild_name,
	attachment_msg_count_t.author_guild_id,
	guilds_t.id as guild_id
from
(
	select 
		count(*) attachment_msg_count,
		author_guild_id
	from attachments_t
    where guild_id = '{guild_id}'
	group by author_guild_id
) as attachment_msg_count_t
join authors_t on attachment_msg_count_t.author_guild_id = authors_t.id
join guilds_t  on authors_t.guild_id = guilds_t.id
order by attachment_msg_count_t.attachment_msg_count desc;
"""

In [ ]:
cursor.execute(query)
ealiest_message_per_guild_results = cursor.fetchall()
df = pd.DataFrame.from_dict(ealiest_message_per_guild_results)

In [ ]:
df.to_csv(output_path + "/Who sent the most attachments?.csv")

In [ ]:
df

## What discord author edits the highest percentage of their messages within a particular discord guild?

In [ ]:
query = f"""
select
	authors_t.author_name,
	authors_t.nickname,
	msg_timestamp_edited_count_t.msg_timestamp_edited_count,
	author_msg_count_t.msg_count,
	msg_timestamp_edited_count_t.msg_timestamp_edited_count::FLOAT  / author_msg_count_t.msg_count::FLOAT  * 100 as msg_edited_percentage,
	msg_timestamp_edited_count_t.author_guild_id,
	guilds_t.guild_name,
	guilds_t.id as guild_id
from
(
	select 
		count(*) as msg_timestamp_edited_count,
		author_guild_id
	from 
		messages_t
	where
		msg_timestampedited is not null -- TO CHANGE
        and guild_id = '{guild_id}'
	group by author_guild_id
) as msg_timestamp_edited_count_t
join (
	select 
		count(*) as msg_count,
		author_guild_id
	from 
		messages_t
    where guild_id = '{guild_id}'
	group by author_guild_id
) as author_msg_count_t on author_msg_count_t.author_guild_id = msg_timestamp_edited_count_t.author_guild_id
join authors_t on msg_timestamp_edited_count_t.author_guild_id = authors_t.id
join guilds_t  on authors_t.guild_id = guilds_t.id
where msg_count > 1
order by msg_edited_percentage desc;
"""

In [ ]:
cursor.execute(query)
ealiest_message_per_guild_results = cursor.fetchall()
df = pd.DataFrame.from_dict(ealiest_message_per_guild_results)

In [ ]:
df.to_csv(output_path + "/What discord author edits the highest percentage of their messages within a particular discord guild.csv")

In [ ]:
df